# Philosophy of Testing

**Testing** is the cornerstone of *scalable* software engineering.  If you **ever** want to contribute to an open source project (*strongly* encouraged if you wanna work in tech), learn to love your tests.  Generally speaking, there are two categories of tests: **unit tests** and **integration tests**.

**Unit tests** compare output of a single function to an expected result, and are *your contract with developers*.  Whenever you write or modify a function, it is *good practice* to provide test cases.  This is *especially* important when your method depends on helper functions which may get updated over time.

**Integration tests** compare the output of an entire application/algorithm, and are *your contract with customers*.  An integration test could make sure a web-page is served and rendered correctly, or it could check that a numerical PDE-solver returns a solution within theoretical limits for a problem with analytic solutions.

As a codebase becomes more complex, the distinction between unit-tests and integration-tests can be fuzzy... For this reason, it is *good practice* to write code with testing in mind.  This is more art than science, and comes with practice and experience.

# Instructions

In this notebook, we have provided **unit tests** for your `classical_gram_schmidt` and `modified_gram_schmidt` implementations.  We have *also* provided a working **implementation** of CGS for reference - ultimately, you should replace our implementation with your own.

First, paste in your implementations of CGS and MGS.  Each function should take a matrix and return a pair of matrices (Q, R).

In [127]:
import numpy as np

def classical_gram_schmidt(A) -> (np.ndarray, np.ndarray):
    """Returns the QR decomposition of A using the classical gram-schmidt algorithm.

    Arguments:
        A (Matrix): A matrix whose columns are linearly independent.
    Returns:
        (Q, R): The QR decomposition of A.
    """
    # This is *my* implementation, provided for reference.
    # You should replace it with your own, but pay attention
    # to the input/output signatures.
    A = np.asanyarray(A, dtype=np.float64)
    m, n = A.shape
    assert m >= n

    Q = np.zeros_like(A)
    for j in range(n):
        qj = A[:, j].copy()
        for k in range(j):
            qj -= (Q[:, k] @ A[:, j]) * Q[:, k]
        Q[:, j] = qj / np.linalg.norm(qj)
    R = Q.T @ A  # This trick *probably* won't cut it for MGS...
    return Q, R

def modified_gram_schmidt(A: np.ndarray) -> (np.ndarray, np.ndarray):
    """Returns the QR decomposition of A using the modified gram-schmidt algorithm.

    Arguments:
        A (np.ndarray): A matrix whose columns are linearly independent.
    Returns:
        (Q, R): The QR decomposition of A.
    """
    # Paste your implementation here!

Next, run the following cell to import some testing libraries.

In [128]:
%pip install parameterized
%pip install ipython_unittest
%reload_ext ipython_unittest

import unittest
import sympy as sp
import numpy as np
from parameterized import parameterized

This final cell contains the promised **unit tests**.  We're using [sympy](https://www.sympy.org/) to compute *exact* QR-decompositions (symbolically), and evaluating these as `float64`.

Notice that we're checking $Q$ for *relative error* (rtol), and $R$ for *absolute error* (atol)...  Hint: this means that `R = Q.T @ A` *probably* won't cut it for MGS.



In [1]:
%%unittest_main

class HW01_Tests(unittest.TestCase):

    @staticmethod
    def _qr_exact(A, dtype=np.float64):
        # Exact *symbolic* solution to the QR decomp.
        Q, R = sp.Matrix(A).QRdecomposition()
        # Numeric solution up to specified dtype.
        return (np.array(Q, dtype), np.array(R, dtype))


    @parameterized.expand([
        # CGS solves this quite well.
        ([[1, 1], [1, 0]], ),
        # But struggles here....
        ([[1000, 1001], [1000, 999]], 3e-13, 1e-9),
        # Make sure to handle m>n!
        ([[1, 0], [0, 1], [1, 1]], ),
    ])
    def test_classical(self, A, qrtol=1e-15, ratol=1e-15):
        Qexpect, Rexpect = self._qr_exact(A)
        Qobserved, Robserved = classical_gram_schmidt(A)
        
        np.testing.assert_allclose(Qexpect, Qobserved, rtol=qrtol)
        np.testing.assert_allclose(Rexpect, Robserved, atol=ratol)


    @parameterized.expand([
        # MGS also handles this.
        ([[1, 1], [1, 0]], ),
        # And does much better here.
        ([[1000, 1001], [1000, 999]], 3e-13),
        # Make sure to handle m>n!
        ([[1, 0], [0, 1], [1, 1]], ),
    ])
    def test_modified(self, A, qrtol=1e-15, ratol=1e-15):
        Qexpect, Rexpect = self._qr_exact(A)
        Qobserved, Robserved = modified_gram_schmidt(A)
        
        np.testing.assert_allclose(Qexpect, Qobserved, rtol=qrtol)
        np.testing.assert_allclose(Rexpect, Robserved, atol=ratol)
